## Data

In [ ]:
import pandas as pd

data = pd.read_csv("src/difference_in_difference_with_deep_learning/data/testdata.csv")

In [ ]:
import yaml

with open(
    "src/difference_in_difference_with_deep_learning/data_management/data_info.yaml",
) as file:
    data_info = yaml.safe_load(file)

In [ ]:
data_select_1 = data[data_info["categorical_columns"][0]]
data_select_2 = data[data_info["categorical_columns"][1]]

In [ ]:
data.loc[:, data_info["causal_effect"]] = data_select_1 * data_select_2

## Summary

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
def did_regression(data, data_info):
    # Extract variables from data_info
    outcome_variable = data_info["outcome"]
    causal_effect_variable = data_info["causal_effect"]
    categorical_columns = data_info["categorical_columns"]
    control_columns = data_info["control_columns"]

    # Create formula string
    formula = f"{outcome_variable} ~ {causal_effect_variable} + {' + '.join(categorical_columns)}  + {' + '.join(control_columns)}"

    # Set up the regression model
    reg_model = smf.ols(formula=formula, data=data)

    # Fit the regression model
    results = reg_model.fit()

    # Return the summary of the regression results
    return results.summary2()

## Summary of how to change summary output for my need (postponed)

In [ ]:
summary = did_regression(data, data_info)
print(summary)

In [ ]:
data0 = summary.tables[0]

In [ ]:
data0

In [ ]:
adj_r_squared_row = data0.loc[data0[0] == "Adj. R-squared:"]
no_observations_row = data0.loc[data0[3] == "No. Observations:"]

In [ ]:
data1 = summary.tables[1]
index = ["Intercept", "interaction", "FQ", "Reform", "Age", "WagePartner"]
summary_df = pd.DataFrame(data, index=index)
summary_df = summary_df.round(3)
print(summary_df)

In [ ]:
summary_df = pd.concat(
    [summary_df, adj_r_squared_row, no_observations_row],
    ignore_index=True,
)

In [ ]:
summary_df

In [ ]:
latex_table = summary_df.to_latex()

print(latex_table)

In [ ]:
for table in summary.tables:
    print(table.as_latex_tabular())

# Groupby richtig machen

In [ ]:
def estimate_regression(data, data_info):
    """Estimate regression models for each time period and summarize the results.

    Parameters:
        data (DataFrame): The dataset containing all variables.
        data_info (dict): Dictionary containing data configuration information.

    Returns:
        DataFrame: A DataFrame containing summary statistics for each time period.
    """
    results_list = []

    for time_period in data[data_info["time"]].unique():
        # Filter the data for the current time period
        data_time_period = data[data[data_info["time"]] == time_period]

        # Define the regression formula
        formula = f"{data_info['outcome']} ~ {data_info['causal_effect']} + {' + '.join(data_info['categorical_columns'] )} + {' + '.join(data_info['control_columns'])}"

        # Fit the regression model
        reg_model = smf.ols(formula=formula, data=data_time_period)
        results = reg_model.fit()

        # Extract coefficient, std. error, and p-value
        coefficient = results.params[data_info["causal_effect"]]
        std_error = results.bse[data_info["causal_effect"]]
        p_value = results.pvalues[data_info["causal_effect"]]

        # Calculate the control mean
        control_mean = data_time_period[data_info["outcome"]].mean()

        # Calculate the difference between treatment and control groups
        difference_tc = (
            data_time_period[data_time_period[data_info["causal_effect"]] == 1][
                data_info["outcome"]
            ].mean()
            - control_mean
        )

        # Calculate the difference with controls
        # Assume mean of interaction term for simplicity
        data_time_period[data_info["causal_effect"]].mean()
        difference_tc_controls = (
            difference_tc
            - coefficient
            * (
                data_time_period[data_info["control_columns"]]
                - data_time_period[data_info["control_columns"]].mean()
            )
            .mean()
            .sum()
        )

        # Append the results to the list
        results_list.append(
            {
                "Time Period": time_period,
                "Control Mean": control_mean,
                "Difference T-C": difference_tc,
                "Difference T-C with Controls": difference_tc_controls,
                "Coefficient": coefficient,
                "Std. Error": std_error,
                "P-value": p_value,
            },
        )

    # Convert the list of dictionaries to a DataFrame
    return pd.DataFrame(results_list)

In [ ]:
result_df = estimate_regression(data, data_info)

In [ ]:
result_df

In [ ]:
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

time_order = ["t-2", "t-1", "t+1", "t+2", "t+3"]

# Convert Time column to categorical with defined order
df["time"] = pd.Categorical(df["time"], categories=time_order, ordered=True)


# Calculate average wage for each group over time
grouped = df.groupby(["FQ", "Reform", "time"])["wage_year"].mean().reset_index()

# Separate data for the two groups
group0 = grouped[(grouped["FQ"] == 0) & (grouped["Reform"] == 0)]
group1 = grouped[(grouped["FQ"] == 1) & (grouped["Reform"] == 1)]

# Calculate counterfactual average wage for each time period
counterfactual = group0.copy()
counterfactual["wage_year"] += group1["wage_year"].mean() - group0["wage_year"].mean()


# Plotting
plt.figure(figsize=(10, 5))
plt.plot(group0["time"], group0["wage_year"], label="Group 0", marker="o")
plt.plot(group1["time"], group1["wage_year"], label="Group 1", marker="o")
plt.plot(
    counterfactual["time"],
    counterfactual["wage_year"],
    label="Counterfactual",
    linestyle="--",
    marker="o",
)


# Add labels and legend
plt.xlabel("Time")
plt.ylabel("Average Wage")
plt.title("Difference-in-Differences Plot")
plt.legend()
plt.grid(True)

# Show plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# deep learning estimation

In [ ]:
import sys
import time

import matplotlib.pyplot as plt

# Stopping Tensorflow from printing info messages and warnings.
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf

FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_boolean("update", False, """Record the simulation results.""")
tf.flags.DEFINE_boolean("plot_true", False, """Show plots.""")
tf.flags.DEFINE_boolean("verbose", True, """Show detailed messages.""")
tf.flags.DEFINE_integer("nsimulations", 1, """How many simulations to run.""")
tf.flags.DEFINE_integer(
    "nconsumer_characteristics",
    100,
    """Number of consumer characteristics.""",
)
tf.flags.DEFINE_string(
    "treatment",
    "not_random",
    """Are customers treated at random or not.""",
)
tf.flags.DEFINE_string(
    "model",
    "quadratic",
    """Is the mapping from consumer characteristics
                       to their preferences linear or quadratic.""",
)
tf.flags.DEFINE_string(
    "architecture",
    "architecture_1_",
    """Which NN architecture to use.""",
)
tf.flags.DEFINE_integer("data_seed", None, """Seed to use to create fake data.""")

# Manually set sys.argv to simulate command-line invocation
sys.argv = [sys.argv[0]]

# Parse flags
FLAGS(sys.argv)

# Access remaining arguments after parsing flags
remaining_args = [arg for arg in sys.argv[1:] if arg.startswith("--")]
assert remaining_args == []

In [ ]:
# Different architectures for the first NN
if FLAGS.architecture == "architecture_1_":
    hidden_layer_sizes = [20, 10, 5]
    dropout_rates_train = [0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "none"]

elif FLAGS.architecture == "architecture_2_":
    hidden_layer_sizes = [60, 30, 20]
    dropout_rates_train = [0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "none"]

elif FLAGS.architecture == "architecture_3_":
    hidden_layer_sizes = [80, 80, 80]
    dropout_rates_train = [0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "none"]

elif FLAGS.architecture == "architecture_4_":
    hidden_layer_sizes = [20, 15, 10, 5]
    activation_functions = ["relu", "relu", "relu", "relu", "none"]
    dropout_rates_train = [0, 0, 0, 0, 0]

elif FLAGS.architecture == "architecture_5_":
    hidden_layer_sizes = [60, 30, 20, 10]
    activation_functions = ["relu", "relu", "relu", "relu", "none"]
    dropout_rates_train = [0, 0, 0, 0, 0]

elif FLAGS.architecture == "architecture_6_":
    hidden_layer_sizes = [80, 80, 80, 80]
    activation_functions = ["relu", "relu", "relu", "relu", "none"]
    dropout_rates_train = [0, 0, 0, 0, 0]

elif FLAGS.architecture == "architecture_7_":
    hidden_layer_sizes = [20, 15, 15, 10, 10, 5]
    dropout_rates_train = [0, 0, 0, 0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "relu", "relu", "relu", "none"]

elif FLAGS.architecture == "architecture_8_":
    hidden_layer_sizes = [60, 30, 20, 20, 10, 5]
    dropout_rates_train = [0, 0, 0, 0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "relu", "relu", "relu", "none"]

elif FLAGS.architecture == "architecture_9_":
    hidden_layer_sizes = [80, 80, 80, 80, 80, 80]
    dropout_rates_train = [0, 0, 0, 0, 0, 0, 0]
    activation_functions = ["relu", "relu", "relu", "relu", "relu", "relu", "none"]
else:
    msg = "Architecture not found! Check the spelling."
    raise ValueError(msg)

if FLAGS.nconsumer_characteristics < 20:
    raise ValueError(
        "Number of consumer characteristics " + "should not be less than 20.",
    )

In [ ]:
dropout_rates_test = [0 for i in dropout_rates_train]

# Architecture for the second NN that estimates
# propensity scores
hidden_layer_sizes_treatment = [50, 30]
activation_functions_treatment = ["relu", "relu", "none"]
dropout_rates_train_treatment = [0, 0, 0]
dropout_rates_test_treatment = [0 for i in dropout_rates_train_treatment]

# Setting parameters values for generating fake data
nconsumers = 10000

# Run parameters
train_proportion = 0.9
max_nepochs = 5000
max_epochs_without_change = 30

early_stopping = train_proportion != 1

optimizer = "Adam"
learning_rate = 0.009
batch_size = 128
batch_size_t = None

# Regularization parameters
alpha = 0.0
r = 0.2

# Checking for spelling errors
if not (FLAGS.model == "quadratic" or FLAGS.model == "simple"):
    msg = "Check whether model type is spelled correctly!"
    raise ValueError(msg)
if not (FLAGS.treatment == "random" or FLAGS.treatment == "not_random"):
    msg = "Check whether treatment type is spelled correctly!"
    raise ValueError(msg)

start_time = time.time()

X_train = T_train = Y_train = X_valid = T_valid = Y_valid = X = T_real = Y = None

In [ ]:
def get_train_test_inds(t):
    """Split the dataset into training and validation sets while
    preserving the proportion of targeted customers in both datasets.

    Inputs:
    -------
        t: array-like, shape=(N, 1)
            Treatment array.
    Outputs:
    -------
        train_inds: array of bools
            Indices of the training set.
        valid_inds: array of bools
            Indices of the validation set.
    """
    t_array = np.array(t)
    train_inds = np.zeros(len(t_array), dtype=bool)
    valid_inds = np.zeros(len(t_array), dtype=bool)
    values = np.unique(t_array)
    for value in values:
        value_inds = np.nonzero(t_array == value)[0]
        np.random.shuffle(value_inds)
        n = int(train_proportion * len(value_inds))
        train_inds[value_inds[:n]] = True
        valid_inds[value_inds[n:]] = True
    return train_inds, valid_inds